In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt, timedelta as td

import json
import random
import string

from copy import deepcopy as dc
import os

from TSFEFS import *


# Read Test Plan
# 1. Read from a .tsfefs folder with df_index file missing.
# 2. Read from a .tsfefs folder with an empty df_index file.
# 3. Read from a .tsfefs folder with non-empty df_index, but without pieces.
# 4. Read from a .tsfefs folder with non-empty df_index with pieces.


# 1. Read from a .tsfefs folder with df_index file missing.


In [2]:
dict_meta = {
    "piece_name_len": 8,
    "time_col": "time", "datetime_format": "%Y-%m-%d %H:%M:%S", 
    "max_row_per_piece": 400000,
    "colnames":["time"],
    "cache_config":{"rows_in_cache":None,"len_of_cache":3}
}


In [3]:
path = os.getcwd()
name = "read1"
fullpath = "%s/%s.%s"%(path,name,TSFEFS.extension)
if not os.path.isdir(fullpath):
    os.mkdir(fullpath)

json.dump(dict_meta, open("%s/%s"%(fullpath,TSFEFS.meta_json_name),'w'))


In [4]:
tsfefs_read1 = TSFEFS()
try:
    tsfefs_read1.read(fullpath)
except:
    print("Can't read without the index file")
    print("Remove folder %s"%fullpath)
    for f in os.listdir(fullpath):
        os.remove("%s/%s"%(fullpath,f))
    os.rmdir(fullpath)    


Can't read without the index file
Remove folder /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS/read1.tsfefs


# 2. Read from a .tsfefs folder with an empty df_index file.

In [5]:
df_index = pd.DataFrame({"piece":[],"type":[],"fr":[],"to":[],"row_cnt":[]})


In [6]:
path = os.getcwd()
name = "read2"
fullpath = "%s/%s.%s"%(path,name,TSFEFS.extension)
if not os.path.isdir(fullpath):
    os.mkdir(fullpath)

json.dump(dict_meta, open("%s/%s"%(fullpath,TSFEFS.meta_json_name),'w'))
df_index.to_csv("%s/%s"%(fullpath,TSFEFS.index_df_name), index=False)


In [7]:
tsfefs_read2 = TSFEFS()
try:
    tsfefs_read2.read(fullpath)
except:
    print("Can't read without the index file")
    print("Remove folder %s"%fullpath)
    for f in os.listdir(fullpath):
        os.remove("%s/%s"%(fullpath,f))
    os.rmdir(fullpath)    


In [8]:
tsfefs_read2.print_tsfefs_info()


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: read2
pieces: []
fr: None , to: None
frs: []
tos: []
types: []
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: None
row_cnts: []
actions: [] , action_params: []
cache: [] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


In [9]:
tsfefs_read2.remove()
del tsfefs_read2; tsfefs_read2 = None


# 3. Read from a .tsfefs folder with non-empty df_index, but without pieces.


In [10]:
fr, to = "2020-01-01 00:00:00", "2020-01-02 00:00:00"
fr = dt.strptime(fr,dict_meta["datetime_format"])
to = dt.strptime(to,dict_meta["datetime_format"])
ts_gap = (to - fr)
seconds = ts_gap.seconds + ts_gap.days*24*60*60
# print(seconds)

df_index = pd.DataFrame({"piece":["abcdefgh"],"type":["csv"],"fr":["2020-01-01 00:00:00"],"to":["2020-01-02 00:00:00"],"row_cnt":[seconds+1]})


In [11]:
path = os.getcwd()
name = "read3"
fullpath = "%s/%s.%s"%(path,name,TSFEFS.extension)
if not os.path.isdir(fullpath):
    os.mkdir(fullpath)

json.dump(dict_meta, open("%s/%s"%(fullpath,TSFEFS.meta_json_name),'w'))
df_index.to_csv("%s/%s"%(fullpath,TSFEFS.index_df_name), index=False)


In [12]:
tsfefs_read3 = TSFEFS()
try:
    tsfefs_read3.read(fullpath)
except:
    print("Can't read without the index file")
    print("Remove folder %s"%fullpath)
    for f in os.listdir(fullpath):
        os.remove("%s/%s"%(fullpath,f))
    os.rmdir(fullpath)


In [13]:
tsfefs_read3.print_tsfefs_info()


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: read3
pieces: ['abcdefgh']
fr: 2020-01-01 00:00:00 , to: 2020-01-02 00:00:00
frs: ['2020-01-01 00:00:00']
tos: ['2020-01-02 00:00:00']
types: ['csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 86401
row_cnts: [86401]
actions: [''] , action_params: [None]
cache: [] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


#### What will happen when accessing "time" column without the piece?

In [14]:
try:
    print(tsfefs_read3["time"])
except:
    print("Error because the file piece doesn't exist.")
    print("Remove folder %s"%fullpath)
    for f in os.listdir(fullpath):
        os.remove("%s/%s"%(fullpath,f))
    os.rmdir(fullpath)


Error because the file piece doesn't exist.
Remove folder /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS/read3.tsfefs


# 4. Read from a .tsfefs folder with non-empty df_index with pieces.

In [15]:
tss = [ fr + td(seconds=i) for i in range(seconds+1) ]
print(min(tss),max(tss))
df = pd.DataFrame({dict_meta["time_col"]:tss})


2020-01-01 00:00:00 2020-01-02 00:00:00


In [16]:
path = os.getcwd()
name = "read4"
fullpath = "%s/%s.%s"%(path,name,TSFEFS.extension)
if not os.path.isdir(fullpath):
    os.mkdir(fullpath)

json.dump(dict_meta, open("%s/%s"%(fullpath,TSFEFS.meta_json_name),'w'))
df_index.to_csv("%s/%s"%(fullpath,TSFEFS.index_df_name), index=False)
df.to_csv("%s/%s"%(fullpath,"abcdefgh"),index=False)


In [17]:
tsfefs_read4 = TSFEFS()
try:
    tsfefs_read4.read(fullpath)
except:
    print("Can't read without the index file")
    print("Remove folder %s"%fullpath)
    for f in os.listdir(fullpath):
        os.remove("%s/%s"%(fullpath,f))
    os.rmdir(fullpath)


In [18]:
try:
    print(tsfefs_read4["time"])
except:
    print("Error because the file piece doesn't exist.")
    print("Remove folder %s"%fullpath)
    for f in os.listdir(fullpath):
        os.remove("%s/%s"%(fullpath,f))
    os.rmdir(fullpath)


0       2020-01-01 00:00:00
1       2020-01-01 00:00:01
2       2020-01-01 00:00:02
3       2020-01-01 00:00:03
4       2020-01-01 00:00:04
                ...        
86396   2020-01-01 23:59:56
86397   2020-01-01 23:59:57
86398   2020-01-01 23:59:58
86399   2020-01-01 23:59:59
86400   2020-01-02 00:00:00
Name: time, Length: 86401, dtype: datetime64[ns]


In [19]:
tsfefs_read4.remove()


# Write Test Strategy: 
## - base tsfefs instance create by load() and import(),
## - copy to each case,
## - try various write().


In [20]:
tsfefs_base = TSFEFS.create(dict_meta, "base")
tsfefs_base.import_dataframe(df)
tsfefs_base.take_actions(max_level=0)
tsfefs_base.print_tsfefs_info()


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: base
pieces: ['hbngiumq']
fr: 2020-01-01 00:00:00 , to: 2020-01-02 00:00:00
frs: ['2020-01-01 00:00:00']
tos: ['2020-01-02 00:00:00']
types: ['csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 86401
row_cnts: [86401]
actions: ['save'] , action_params: [None]
cache: [0] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


# Write Test Plan
# 1. Clone and write as write1.
# 2. Clone, add a non-conflicting dataframe, and write as write2.
# 3. Clone, add a conflicting dataframe, and write as write3.
# 4. Clone, add a non-conflicting tsfefs_, and write as write4.
# 5. Clone, add a conflicting tsfefs_, and write as write5.

# 1. Clone and write as write1.


In [21]:
tsfefs_write1 = tsfefs_base.clone(tsfefs_base.path, "write1")
tsfefs_write1.print_tsfefs_info()


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write1
pieces: ['hbngiumq']
fr: 2020-01-01 00:00:00 , to: 2020-01-02 00:00:00
frs: ['2020-01-01 00:00:00']
tos: ['2020-01-02 00:00:00']
types: ['csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 86401
row_cnts: [86401]
actions: ['save'] , action_params: [None]
cache: [0] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


In [22]:
fullpath = "%s/%s.%s"%(tsfefs_write1.path,tsfefs_write1.name,TSFEFS.extension)
try:
    tsfefs_write1.write(fullpath)
except:
    print("Actions not cleared, cannot write.")


Actions not cleared, cannot write.


In [23]:
tsfefs_write1.take_actions(max_level=4)
tsfefs_write1.write(fullpath)
tsfefs_write1.print_tsfefs_info()
del tsfefs_write1; tsfefs_write1 = None


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write1
pieces: ['hbngiumq']
fr: 2020-01-01 00:00:00 , to: 2020-01-02 00:00:00
frs: ['2020-01-01 00:00:00']
tos: ['2020-01-02 00:00:00']
types: ['csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 86401
row_cnts: [86401]
actions: [''] , action_params: [None]
cache: [0] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


#### check by reading it again.

In [24]:
tsfefs_write1 = TSFEFS()
tsfefs_write1.read(fullpath)
print(tsfefs_write1[tsfefs_write1.time_col])
print()
tsfefs_write1.print_tsfefs_info()
tsfefs_write1.remove()


0       2020-01-01 00:00:00
1       2020-01-01 00:00:01
2       2020-01-01 00:00:02
3       2020-01-01 00:00:03
4       2020-01-01 00:00:04
                ...        
86396   2020-01-01 23:59:56
86397   2020-01-01 23:59:57
86398   2020-01-01 23:59:58
86399   2020-01-01 23:59:59
86400   2020-01-02 00:00:00
Name: time, Length: 86401, dtype: datetime64[ns]

path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write1
pieces: ['hbngiumq']
fr: 2020-01-01 00:00:00 , to: 2020-01-02 00:00:00
frs: ['2020-01-01 00:00:00']
tos: ['2020-01-02 00:00:00']
types: ['csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 86401
row_cnts: [86401]
actions: [''] , action_params: [None]
cache: [] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


# 2. Clone, add a non-conflicting dataframe, and write as write2.


In [25]:
tss2 = [ ts + td(days=1) for ts in tss ]
print(min(tss2),max(tss2))
df2 = pd.DataFrame({dict_meta["time_col"]:tss2})


2020-01-02 00:00:00 2020-01-03 00:00:00


In [26]:
tsfefs_write2 = tsfefs_base.clone(tsfefs_base.path, "write2")
tsfefs_write2 += df2
tsfefs_write2.take_actions(max_level=0)
tsfefs_write2.print_tsfefs_info()


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write2
pieces: ['hbngiumq', 'ccuvspxr']
fr: 2020-01-01 00:00:00 , to: 2020-01-03 00:00:00
frs: ['2020-01-01 00:00:00', '2020-01-02 00:00:00']
tos: ['2020-01-02 00:00:00', '2020-01-03 00:00:00']
types: ['csv', 'csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 172802
row_cnts: [86401, 86401]
actions: ['save', 'save'] , action_params: [None, None]
cache: [1, 0] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


In [27]:
tsfefs_write2.take_actions(max_level=4)
fullpath = "%s/%s.%s"%(tsfefs_write2.path,tsfefs_write2.name,TSFEFS.extension)
tsfefs_write2.write(fullpath)
tsfefs_write2.print_tsfefs_info()
del tsfefs_write2; tsfefs_write2 = None


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write2
pieces: ['hbngiumq', 'ccuvspxr']
fr: 2020-01-01 00:00:00 , to: 2020-01-03 00:00:00
frs: ['2020-01-01 00:00:00', '2020-01-02 00:00:00']
tos: ['2020-01-02 00:00:00', '2020-01-03 00:00:00']
types: ['csv', 'csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 172802
row_cnts: [86401, 86401]
actions: ['', ''] , action_params: [None, None]
cache: [1, 0] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


#### check by reading it again.

In [28]:
tsfefs_write2 = TSFEFS()
tsfefs_write2.read(fullpath)
print(tsfefs_write2[tsfefs_write2.time_col])
print()
tsfefs_write2.print_tsfefs_info()
tsfefs_write2.remove()


0        2020-01-01 00:00:00
1        2020-01-01 00:00:01
2        2020-01-01 00:00:02
3        2020-01-01 00:00:03
4        2020-01-01 00:00:04
                 ...        
172797   2020-01-02 23:59:56
172798   2020-01-02 23:59:57
172799   2020-01-02 23:59:58
172800   2020-01-02 23:59:59
172801   2020-01-03 00:00:00
Name: time, Length: 172802, dtype: datetime64[ns]

path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write2
pieces: ['hbngiumq', 'ccuvspxr']
fr: 2020-01-01 00:00:00 , to: 2020-01-03 00:00:00
frs: ['2020-01-01 00:00:00', '2020-01-02 00:00:00']
tos: ['2020-01-02 00:00:00', '2020-01-03 00:00:00']
types: ['csv', 'csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 172802
row_cnts: [86401, 86401]
actions: ['', ''] , action_params: [None, None]
cache: [] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


# 3. Clone, add a conflicting dataframe, and write as write3.


In [29]:
tss3 = [ ts + td(days=0.5) for ts in tss ]
print(min(tss3),max(tss3))
df3 = pd.DataFrame({dict_meta["time_col"]:tss3})


2020-01-01 12:00:00 2020-01-02 12:00:00


In [30]:
tsfefs_write3 = tsfefs_base.clone(tsfefs_base.path, "write3")
tsfefs_write3 += df3
tsfefs_write3.take_actions(max_level=0)
tsfefs_write3.print_tsfefs_info()


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write3
pieces: ['hbngiumq', 'gzliayqf']
fr: 2020-01-01 00:00:00 , to: 2020-01-02 12:00:00
frs: ['2020-01-01 00:00:00', '2020-01-01 12:00:00']
tos: ['2020-01-02 00:00:00', '2020-01-02 12:00:00']
types: ['csv', 'csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 172802
row_cnts: [86401, 86401]
actions: ['save', 'save'] , action_params: [None, None]
cache: [1, 0] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


In [31]:
tsfefs_write3.take_actions(max_level=4)
fullpath = "%s/%s.%s"%(tsfefs_write3.path,tsfefs_write3.name,TSFEFS.extension)
tsfefs_write3.write(fullpath)
tsfefs_write3.print_tsfefs_info()
del tsfefs_write3; tsfefs_write3 = None


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write3
pieces: ['hbngiumq', 'gzliayqf']
fr: 2020-01-01 00:00:00 , to: 2020-01-02 12:00:00
frs: ['2020-01-01 00:00:00', '2020-01-01 12:00:00']
tos: ['2020-01-02 00:00:00', '2020-01-02 12:00:00']
types: ['csv', 'csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 172802
row_cnts: [86401, 86401]
actions: ['', ''] , action_params: [None, None]
cache: [1, 0] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


#### check by reading it again.

In [32]:
tsfefs_write3 = TSFEFS()
tsfefs_write3.read(fullpath)

try:
    print(tsfefs_write3[tsfefs_write3.time_col])
    print()
    tsfefs_write3.print_tsfefs_info()
    tsfefs_write3.remove()
except:
    print("Conclusion: can be saved with conflicting times, but must resolve_conflict() after read and before access.")
    print()
    tsfefs_write3.resolve_conflict()
    print(tsfefs_write3[tsfefs_write3.time_col])
    print()
    tsfefs_write3.print_tsfefs_info()
    tsfefs_write3.remove()


Conclusion: can be saved with conflicting times, but must resolve_conflict() after read and before access.

0        2020-01-01 00:00:00
1        2020-01-01 00:00:01
2        2020-01-01 00:00:02
3        2020-01-01 00:00:03
4        2020-01-01 00:00:04
                 ...        
172797   2020-01-02 11:59:56
172798   2020-01-02 11:59:57
172799   2020-01-02 11:59:58
172800   2020-01-02 11:59:59
172801   2020-01-02 12:00:00
Name: time, Length: 172802, dtype: datetime64[ns]

path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write3
pieces: ['rfothaet', 'qtptrbec', 'mevvhsik']
fr: 2020-01-01 00:00:00 , to: 2020-01-02 12:00:00
frs: ['2020-01-01 00:00:00', '2020-01-02 00:00:01', '2020-01-01 12:00:00']
tos: ['2020-01-01 11:59:59', '2020-01-02 12:00:00', '2020-01-02 00:00:00']
types: ['csv', 'csv', 'csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 172802
row_cnts: [43200, 43200, 86402]
actions: ['', '', '

# 4. Clone, add a non-conflicting tsfefs_, and write as write4.


In [33]:
# This is the tsfefs to be added
tsfefs_ = TSFEFS.create(dict_meta, "_")
tsfefs_.import_dataframe(df2)
tsfefs_.take_actions(max_level=0)
tsfefs_.print_tsfefs_info()


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: _
pieces: ['qwamvhyu']
fr: 2020-01-02 00:00:00 , to: 2020-01-03 00:00:00
frs: ['2020-01-02 00:00:00']
tos: ['2020-01-03 00:00:00']
types: ['csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 86401
row_cnts: [86401]
actions: ['save'] , action_params: [None]
cache: [0] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


In [34]:
tsfefs_write4 = tsfefs_base.clone(tsfefs_base.path, "write4")
tsfefs_write4 += tsfefs_
tsfefs_write4.take_actions(max_level=0)
tsfefs_write4.print_tsfefs_info()


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write4
pieces: ['hbngiumq', '_']
fr: 2020-01-01 00:00:00 , to: 2020-01-03 00:00:00
frs: ['2020-01-01 00:00:00', '2020-01-02 00:00:00']
tos: ['2020-01-02 00:00:00', '2020-01-03 00:00:00']
types: ['csv', 'tsfefs']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 172802
row_cnts: [86401, 86401]
actions: ['save', 'save'] , action_params: [None, None]
cache: [1, 0] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


In [35]:
tsfefs_write4.take_actions(max_level=4)
fullpath = "%s/%s.%s"%(tsfefs_write4.path,tsfefs_write4.name,TSFEFS.extension)
tsfefs_write4.write(fullpath)
tsfefs_write4.print_tsfefs_info()
del tsfefs_write4; tsfefs_write4 = None


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write4
pieces: ['hbngiumq', '_']
fr: 2020-01-01 00:00:00 , to: 2020-01-03 00:00:00
frs: ['2020-01-01 00:00:00', '2020-01-02 00:00:00']
tos: ['2020-01-02 00:00:00', '2020-01-03 00:00:00']
types: ['csv', 'tsfefs']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 172802
row_cnts: [86401, 86401]
actions: ['', ''] , action_params: [None, None]
cache: [1, 0] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


#### check by reading it again.

In [36]:
tsfefs_write4 = TSFEFS()
tsfefs_write4.read(fullpath)

print(tsfefs_write4[tsfefs_write4.time_col])
print()
tsfefs_write4.print_tsfefs_info()
tsfefs_write4.remove()


0        2020-01-01 00:00:00
1        2020-01-01 00:00:01
2        2020-01-01 00:00:02
3        2020-01-01 00:00:03
4        2020-01-01 00:00:04
                 ...        
172797   2020-01-02 23:59:56
172798   2020-01-02 23:59:57
172799   2020-01-02 23:59:58
172800   2020-01-02 23:59:59
172801   2020-01-03 00:00:00
Name: time, Length: 172802, dtype: datetime64[ns]

path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write4
pieces: ['hbngiumq', '_']
fr: 2020-01-01 00:00:00 , to: 2020-01-03 00:00:00
frs: ['2020-01-01 00:00:00', '2020-01-02 00:00:00']
tos: ['2020-01-02 00:00:00', '2020-01-03 00:00:00']
types: ['csv', 'tsfefs']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 172802
row_cnts: [86401, 86401]
actions: ['', ''] , action_params: [None, None]
cache: [] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


# 5. Clone, add a conflicting tsfefs_, and write as write5. 

In [37]:
# This is the tsfefs to be added
tsfefs_ = TSFEFS.create(dict_meta, "_")
tsfefs_.import_dataframe(df3)
tsfefs_.take_actions(max_level=0)
tsfefs_.print_tsfefs_info()


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: _
pieces: ['pgsbernc']
fr: 2020-01-01 12:00:00 , to: 2020-01-02 12:00:00
frs: ['2020-01-01 12:00:00']
tos: ['2020-01-02 12:00:00']
types: ['csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 86401
row_cnts: [86401]
actions: ['save'] , action_params: [None]
cache: [0] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


In [38]:
tsfefs_write5 = tsfefs_base.clone(tsfefs_base.path, "write5")
tsfefs_write5 += tsfefs_
tsfefs_write5.take_actions(max_level=0)
tsfefs_write5.print_tsfefs_info()


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write5
pieces: ['hbngiumq', '_']
fr: 2020-01-01 00:00:00 , to: 2020-01-02 12:00:00
frs: ['2020-01-01 00:00:00', '2020-01-01 12:00:00']
tos: ['2020-01-02 00:00:00', '2020-01-02 12:00:00']
types: ['csv', 'tsfefs']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 172802
row_cnts: [86401, 86401]
actions: ['save', 'save'] , action_params: [None, None]
cache: [1, 0] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


In [39]:
tsfefs_write5.take_actions(max_level=4)
fullpath = "%s/%s.%s"%(tsfefs_write5.path,tsfefs_write5.name,TSFEFS.extension)
tsfefs_write5.write(fullpath)
tsfefs_write5.print_tsfefs_info()
del tsfefs_write5; tsfefs_write5 = None


path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write5
pieces: ['hbngiumq', '_']
fr: 2020-01-01 00:00:00 , to: 2020-01-02 12:00:00
frs: ['2020-01-01 00:00:00', '2020-01-01 12:00:00']
tos: ['2020-01-02 00:00:00', '2020-01-02 12:00:00']
types: ['csv', 'tsfefs']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 172802
row_cnts: [86401, 86401]
actions: ['', ''] , action_params: [None, None]
cache: [1, 0] , cache_config: {'rows_in_cache': None, 'len_of_cache': 3}


#### check by reading it again.

In [40]:
tsfefs_write5 = TSFEFS()
tsfefs_write5.read(fullpath)

try:
    print(tsfefs_write5[tsfefs_write5.time_col])
    print()
    tsfefs_write5.print_tsfefs_info()
    tsfefs_write5.remove()
except:
    print("Conclusion: can be saved with conflicting times, but must resolve_conflict() after read and before access.")
    print()
    tsfefs_write5.resolve_conflict()
    print(tsfefs_write5[tsfefs_write5.time_col])
    print()
    tsfefs_write5.print_tsfefs_info()
    tsfefs_write5.remove()


Conclusion: can be saved with conflicting times, but must resolve_conflict() after read and before access.

0        2020-01-01 00:00:00
1        2020-01-01 00:00:01
2        2020-01-01 00:00:02
3        2020-01-01 00:00:03
4        2020-01-01 00:00:04
                 ...        
172797   2020-01-02 11:59:56
172798   2020-01-02 11:59:57
172799   2020-01-02 11:59:58
172800   2020-01-02 11:59:59
172801   2020-01-02 12:00:00
Name: time, Length: 172802, dtype: datetime64[ns]

path: /Users/yeehinleung/Documents/GreekIsGood/Products/TSFEFS/test TSFEFS , name: write5
pieces: ['tvsxfkbj', 'ougvispi', 'qddfcmix']
fr: 2020-01-01 00:00:00 , to: 2020-01-02 12:00:00
frs: ['2020-01-01 00:00:00', '2020-01-02 00:00:01', '2020-01-01 12:00:00']
tos: ['2020-01-01 11:59:59', '2020-01-02 12:00:00', '2020-01-02 00:00:00']
types: ['csv', 'csv', 'csv']
time_col: time , datetime_format: %Y-%m-%d %H:%M:%S
piece_name_len: 8
colnames: ['time'] , row_cnt: 172802
row_cnts: [43200, 43200, 86402]
actions: ['', '', '